<a href="https://colab.research.google.com/github/Soothysay/Binning/blob/master/Binning_E2E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install plotly

In [ ]:
import numpy as np
import pandas as pd
import math

In [ ]:
def contibin(data, target, bins=10): #Binning Function
  #Empty Dataframe
  newDF,woeDF = pd.DataFrame(), pd.DataFrame()
    
  #Extract Column Names
  cols = data.columns
  for ivars in cols[~cols.isin([target])]:
        if (data[ivars].dtype.kind in 'bifc') and (len(np.unique(data[ivars]))>10):
            binned_x = pd.qcut(data[ivars], bins,  duplicates='drop')
            d0 = pd.DataFrame({'x': binned_x, 'y': data[target]})
            #print(d0)
        else:
            d0 = pd.DataFrame({'x': data[ivars], 'y': data[target]})
        d = d0.groupby("x", as_index=False).agg({"y": ["count", "sum"]})
        d.columns = ['Range', 'Total', 'No. of Good']
        print(d)
        if(type(d['Range'][1])==pd._libs.interval.Interval): #Correction made here
          m=pd.IntervalIndex(d['Range'])
          d=d.drop(['Range'],axis=1)
          d['Lower Limit']=m.left
          d['Upper Limit']=m.right
          d=d[list(('Lower Limit','Upper Limit','Total','No. of Good'))]
        #d=pd.concat([d,d1],axis=0)
        d['No. of Bad'] = d['Total'] - d['No. of Good']
        d['Dist. of Good'] = np.maximum(d['No. of Good'], 0.5) / d['No. of Good'].sum()
        d['Dist. of Bad'] = np.maximum(d['No. of Bad'], 0.5) / d['No. of Bad'].sum()
        d['WoE'] = np.log(d['Dist. of Good']/d['Dist. of Bad'])
        d['IV'] = d['WoE'] * (d['Dist. of Good'] - d['Dist. of Bad'])
        #temp =pd.DataFrame({"Variable" : [ivars], "IV" : [d['IV'].sum()]}, columns = ["Variable", "IV"])
        #newDF=pd.concat([newDF,temp], axis=0)
        woeDF=pd.concat([woeDF,d], axis=0)
        nrow=len(woeDF.index)
        print(type(nrow))
        rows=list()
        i=0
        while(i<nrow):
          rows.append(i+1)
          i=i+1
        print(rows)
        woeDF['Bin Number']=rows
        woeDF['Percentage in Bin']=(woeDF['Total']/woeDF['Total'].sum())*100
        if('Range' in woeDF):
          woeDF=woeDF[list(('Bin Number','Percentage in Bin','Range','Total','No. of Good','No. of Bad','Dist. of Good','Dist. of Bad','WoE','IV'))] #Column structure for categorical values
        else:
          woeDF=woeDF[list(('Bin Number','Percentage in Bin','Lower Limit','Upper Limit','Total','No. of Good','No. of Bad','Dist. of Good','Dist. of Bad','WoE','IV'))] #Column structure for continuous values
  return woeDF

In [ ]:
def merge(df,index1,index2): #Merging for categorical
  name1=df.loc[index1].get(key='Range')
  name2=df.loc[index2].get(key='Range')
  if(type(name1)==pd._libs.interval.Interval):
    m=pd.IntervalIndex([name1,name2])
    d = pd.DataFrame({'left': m.left, 'right': m.right})
    l=d.iloc[0,0]
    r=d.iloc[1,1]
    newname=pd.Interval(left=l,right=r,closed='right')
  else:
    if(type(name1)==np.float64):
      p=int(name1)
      name1=str(p)
    if(type(name2)==np.float64):
      p=int(name2)
      name2=str(p)
    if(type(name1)==int):
      name1=str(name1)
    if(type(name2)==int):
      name2=str(name2)
    newname=name1+"+"+name2
  total1=df.loc[index1].get(key='Total')
  total2=df.loc[index2].get(key='Total')
  newtotal=total1+total2
  good1=df.loc[index1].get(key='No. of Good')
  good2=df.loc[index2].get(key='No. of Good')
  newgood=good1+good2
  bad1=df.loc[index1].get(key='No. of Bad')
  bad2=df.loc[index2].get(key='No. of Bad')
  newbad=bad1+bad2
  newdistgood=newgood/df['No. of Good'].sum()
  newdistbad=newbad/df['No. of Bad'].sum()
  newwoe=math.log((newdistgood/newdistbad))
  newiv=(newdistgood-newdistbad)*newwoe
  newpercent=(newtotal/df['Total'].sum())*100
  df=df.drop([index1,index2])
  newdata=pd.DataFrame({'Bin Number':index1,'Percentage in Bin':newpercent,'Range':newname,'Total':newtotal,'No. of Good':newgood,'No. of Bad':newbad,'Dist. of Good':newdistgood,'Dist. of Bad':newdistbad,'WoE':newwoe,'IV':newiv},
                       index=[index1])
  df = df.append(newdata, ignore_index=False)
  df = df.sort_index().reset_index(drop=True)
  nrow=len(df.index)
  rows=list()
  i=0
  while (i<nrow):
    rows.append(i+1)
    i=i+1
  df['Bin Number']=rows
  df=df[list(('Bin Number','Percentage in Bin','Range','Total','No. of Good','No. of Bad','Dist. of Good','Dist. of Bad','WoE','IV'))]
  return df


In [ ]:
def mergeconti(df,index1,index2): #Merging for continuous
  name1=df.loc[index1].get(key='Lower Limit')
  name2=df.loc[index2].get(key='Upper Limit')
  total1=df.loc[index1].get(key='Total')
  total2=df.loc[index2].get(key='Total')
  newtotal=total1+total2
  good1=df.loc[index1].get(key='No. of Good')
  good2=df.loc[index2].get(key='No. of Good')
  newgood=good1+good2
  bad1=df.loc[index1].get(key='No. of Bad')
  bad2=df.loc[index2].get(key='No. of Bad')
  newbad=bad1+bad2
  newdistgood=newgood/df['No. of Good'].sum()
  newdistbad=newbad/df['No. of Bad'].sum()
  newwoe=math.log((newdistgood/newdistbad))
  newiv=(newdistgood-newdistbad)*newwoe
  newpercent=(newtotal/df['Total'].sum())*100
  df=df.drop([index1,index2])
  newdata=pd.DataFrame({'Bin Number':index1,'Percentage in Bin':newpercent,'Lower Limit':name1,'Upper Limit':name2,'Total':newtotal,'No. of Good':newgood,'No. of Bad':newbad,'Dist. of Good':newdistgood,'Dist. of Bad':newdistbad,'WoE':newwoe,'IV':newiv},
                       index=[index1])
  df = df.append(newdata, ignore_index=False)
  df = df.sort_index().reset_index(drop=True)
  nrow=len(df.index)
  rows=list()
  for i in range(nrow):
    rows.append(i+1)
  df['Bin Number']=rows
  df=df[list(('Bin Number','Percentage in Bin','Lower Limit','Upper Limit','Total','No. of Good','No. of Bad','Dist. of Good','Dist. of Bad','WoE','IV'))]
  return df

In [ ]:
def catfeatureinteger_transdorm (df, woetable, feature):
  nrow=len(df.index)
  woerow=len(woetable.index)
  i=0
  while (i<woerow):
    range=woetable.loc[i].get(key='Range')
    print(range)
    #if (type(range)==int):
    j=0
    npl=list ()
    while (j<nrow):
      if (df.loc[j].get(key=feature)==range):
        npl.append(woetable.loc[i].get(key='IV'))
        #weo=woetable.loc[i].get(key='IV')
        #df.at[j,feature]=weo
      else:
        npl.append(df.loc[j].get(key=feature))
      j=j+1
    df[feature]=npl
    if (type (range)== str):
      p=range.split('+')
      j=0
      while (j<len(p)):
        p[j]=float (p[j])
        j=j+1
      j=0
      while (j<len(p)):
        k=0
        lp=list ()
        while (k<nrow):
          if (df.loc[k].get(key=feature)==p[j]):
            lp.append(woetable.loc[i].get(key='IV'))
            #df.at[k,feature]=woetable.loc[i].get(key='IV')
          else:
            lp.append(df.loc[k].get(key=feature))
          k=k+1
          #print(len(lp))
        df[feature]=lp
        j=j+1
    i=i+1
  return df

In [ ]:
def djangoplot(df):
  import plotly.express as px
  fig = px.bar(df, x='Bin Number', y='WoE',color='Percentage in Bin',labels={'WoE':'Weight of Evidence Value',},title='WOE Table')
  fig.show()

In [ ]:
def catstringfeature_transform(df, woetable, feature):
  nrow=len(df.index)
  woerow=len(woetable.index)
  i=0
  while(i<woerow):
    x=0
    range=woetable.loc[i].get(key='Range')
    if(range.find('+')!=-1):
      x=1
      p=range.split('+')
      k=0
      while (k<len(p)):
        j=0
        while (j<nrow):
          if (df.loc[j].get(key=feature)==p[k]):
            df.at[j,feature]=woetable.loc[i].get(key='IV')
          j=j+1
        k=k+1
    if (x==0):
      j=0
      while (j<nrow):
        if (df.loc[j].get(key=feature)==range):
          df.at[j,feature]=woetable.loc[i].get(key='IV')
        j=j+1
    i=i+1
  return df

In [ ]:
def contifeature_transform(df,woetable,feature): #Transforming continuous features
  nrow=len(df.index)
  woerow=len(woetable.index)
  news=list()
  for i in range(nrow):
    for j in range(woerow):
      if(df.loc[i].get(key=feature)<=woetable.loc[j].get(key='Upper Limit')):
        if(df.loc[i].get(key=feature)>woetable.loc[j].get(key='Lower Limit')):
          news.append(woetable.loc[j].get(key='IV'))
          break
  print(len(news))
  df[feature]=news
  return df

In [ ]:
def main():
  df=pd.read_csv('/content/Churn_Modelling.csv')
  print('The columns are:')
  print(df.columns)
  response=input('Enter the column you wish to choose as response:')
  while (True):
    print('Do you want to bin feature? (Y/N)')
    ch=input('Enter your choice:')
    if(ch=='N'):
      break
    if(ch=='Y'):
      print('The columns are:')
      c1=df.columns
      c1.remove(response)
      print(c1)
      feature=input('Enter the column you wish to choose for binning:')
      df1=df[[feature,response]].copy()
      woetable=contibin(df1,response)
      if (len(np.unique(df[feature]))>10):
        conti=1
      else:
        conti=0
      print('The table is:')
      print(woetable)
      print('The plot is:')
      djangoplot(woetable)
      while (True):
        choi=input('Do You want to merge any 2 consecutive bins? (Y/N)')
        if (choi=='N'):
          break
        if (choi=='Y'):
          bin1=int(input('Enter value of bin1:'))
          bin2=int(input('Enter value of bin2:'))
          index1=min(bin1,bin2)
          index2=max(bin1,bin2)
          if (conti==1):
            woetable=mergeconti(woetable,(index1-1),(index2-1))
          if (conti==0):
            woetable=merge(woetable,(index1-1),(index2-1))
          print('The table is:')
          print(woetable)
          print('The plot is:')
          djangoplot(woetable)
      saver=input('Do you wish to save data? (Y/N)')
      if (saver=='Y'):
        if (conti==1):
          df=contifeature_transform(df,woetable,feature)
        if (conti==0):
          if type (df.loc[0].get(key=feature)) == np.int64:
            df=catfeatureinteger_transdorm (df, woetable, feature)
          else:
            df=catstringfeature_transform(df, woetable, feature)
    print(df)
  return df

In [ ]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [ ]:
df=main()

The columns are:
Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')
Enter the column you wish to choose as response:Exited
Do you want to bin feature? (Y/N)
Enter your choice:Y
The columns are:
Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')
Enter the column you wish to choose for binning:Age
            Range  Total  No. of Good
0  (17.999, 27.0]   1020           73
1    (27.0, 31.0]   1352          108
2    (31.0, 33.0]    860           76
3    (33.0, 35.0]    921           90
4    (35.0, 37.0]    934          115
5    (37.0, 40.0]   1332          224
6    (40.0, 42.0]    687          146
7    (42.0, 46.0]   1009          339
8    (46.0, 5

Do You want to merge any 2 consecutive bins? (Y/N)Y
Enter value of bin1:7
Enter value of bin2:6
The table is:
   Bin Number  Percentage in Bin  Lower Limit  ...  Dist. of Bad       WoE        IV
0           1              10.20       17.999  ...      0.118925 -1.199512  0.099665
1           2              13.52       27.000  ...      0.156223 -1.080628  0.111525
2           3               8.60       31.000  ...      0.098455 -0.970348  0.059332
3           4               9.21       33.000  ...      0.104358 -0.859492  0.051720
4           5               9.34       35.000  ...      0.102851 -0.599824  0.027829
5           6              20.19       37.000  ...      0.207083 -0.131094  0.003335
6           7              10.09       42.000  ...      0.084139  0.682050  0.056120
7           8               9.19       46.000  ...      0.056009  1.422104  0.250568
8           9               9.66       53.000  ...      0.071958  0.986252  0.119310

[9 rows x 11 columns]
The plot is:


Do You want to merge any 2 consecutive bins? (Y/N)N
Do you wish to save data? (Y/N)Y
10000
      RowNumber  CustomerId    Surname  ...  IsActiveMember EstimatedSalary Exited
0             1    15634602   Hargrave  ...               1       101348.88      1
1             2    15647311       Hill  ...               1       112542.58      0
2             3    15619304       Onio  ...               0       113931.57      1
3             4    15701354       Boni  ...               0        93826.63      0
4             5    15737888   Mitchell  ...               1        79084.10      0
...         ...         ...        ...  ...             ...             ...    ...
9995       9996    15606229   Obijiaku  ...               0        96270.64      0
9996       9997    15569892  Johnstone  ...               1       101699.77      0
9997       9998    15584532        Liu  ...               1        42085.58      1
9998       9999    15682355  Sabbatini  ...               0        92888.52    

Do You want to merge any 2 consecutive bins? (Y/N)Y
Enter value of bin1:2
Enter value of bin2:3
The table is:
   Bin Number  Percentage in Bin  ...       WoE       IV
0           1              50.14  ... -0.283430  0.03693
1           2              49.86  ...  0.243747  0.03176

[2 rows x 10 columns]
The plot is:


Do You want to merge any 2 consecutive bins? (Y/N)N
Do you wish to save data? (Y/N)Y
      RowNumber  CustomerId    Surname  ...  IsActiveMember EstimatedSalary Exited
0             1    15634602   Hargrave  ...               1       101348.88      1
1             2    15647311       Hill  ...               1       112542.58      0
2             3    15619304       Onio  ...               0       113931.57      1
3             4    15701354       Boni  ...               0        93826.63      0
4             5    15737888   Mitchell  ...               1        79084.10      0
...         ...         ...        ...  ...             ...             ...    ...
9995       9996    15606229   Obijiaku  ...               0        96270.64      0
9996       9997    15569892  Johnstone  ...               1       101699.77      0
9997       9998    15584532        Liu  ...               1        42085.58      1
9998       9999    15682355  Sabbatini  ...               0        92888.52      1
99

Do You want to merge any 2 consecutive bins? (Y/N)Y
Enter value of bin1:3
Enter value of bin2:4
The table is:
   Bin Number  Percentage in Bin Range  ...  Dist. of Bad       WoE        IV
0           1              50.84     1  ...      0.461509  0.404655  0.093149
1           2              45.90     2  ...      0.532714 -1.137260  0.411545
2           3               3.26   3+4  ...      0.005777  3.169476  0.417358

[3 rows x 10 columns]
The plot is:


Do You want to merge any 2 consecutive bins? (Y/N)Y
Enter value of bin1:2
Enter value of bin2:3
The table is:
   Bin Number  Percentage in Bin  Range  ...  Dist. of Bad       WoE        IV
0           1              50.84      1  ...      0.461509  0.404655  0.093149
1           2              49.16  2+3+4  ...      0.538491 -0.557708  0.128381

[2 rows x 10 columns]
The plot is:


Do You want to merge any 2 consecutive bins? (Y/N)N
Do you wish to save data? (Y/N)Y
1
2+3+4
      RowNumber  CustomerId    Surname  ...  IsActiveMember EstimatedSalary Exited
0             1    15634602   Hargrave  ...               1       101348.88      1
1             2    15647311       Hill  ...               1       112542.58      0
2             3    15619304       Onio  ...               0       113931.57      1
3             4    15701354       Boni  ...               0        93826.63      0
4             5    15737888   Mitchell  ...               1        79084.10      0
...         ...         ...        ...  ...             ...             ...    ...
9995       9996    15606229   Obijiaku  ...               0        96270.64      0
9996       9997    15569892  Johnstone  ...               1       101699.77      0
9997       9998    15584532        Liu  ...               1        42085.58      1
9998       9999    15682355  Sabbatini  ...               0        92888.52  

In [ ]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,0.0369304,Female,0.003335,2,0.00,0.093149,1,1,101348.88,1
1,2,15647311,Hill,608,0.0317598,Female,0.003335,1,83807.86,0.093149,0,1,112542.58,0
2,3,15619304,Onio,502,0.0369304,Female,0.003335,8,159660.80,0.128381,1,0,113931.57,1
3,4,15701354,Boni,699,0.0369304,Female,0.003335,1,0.00,0.128381,0,0,93826.63,0
4,5,15737888,Mitchell,850,0.0317598,Female,0.056120,2,125510.82,0.093149,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,0.0369304,Male,0.003335,5,0.00,0.128381,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,0.0369304,Male,0.051720,10,57369.61,0.093149,1,1,101699.77,0
9997,9998,15584532,Liu,709,0.0369304,Female,0.027829,7,0.00,0.093149,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,0.0317598,Male,0.003335,3,75075.31,0.128381,1,0,92888.52,1


In [ ]:
df=pd.read_csv('/content/Churn_Modelling.csv')
df1=df[['Tenure','Exited']].copy()
woetable=contibin(df1,'Exited')
woetable

           Range  Total  No. of Good
0  (-0.001, 1.0]   1448          327
1     (1.0, 2.0]   1048          201
2     (2.0, 3.0]   1009          213
3     (3.0, 4.0]    989          203
4     (4.0, 5.0]   1012          209
5     (5.0, 6.0]    967          196
6     (6.0, 7.0]   1028          177
7     (7.0, 8.0]   1025          197
8     (8.0, 9.0]    984          213
9    (9.0, 10.0]    490          101
<class 'int'>
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


,Bin Number,Percentage in Bin,Lower Limit,Upper Limit,Total,No. of Good,No. of Bad,Dist. of Good,Dist. of Bad,WoE,IV
0,1,14.48,-0.001,1.0,1448,327,1121,0.160530,0.140776,0.131311,0.002594
1,2,10.48,1.000,2.0,1048,201,847,0.098675,0.106367,-0.075068,0.000577
2,3,10.09,2.000,3.0,1009,213,796,0.104566,0.099962,0.045021,0.000207
3,4,9.89,3.000,4.0,989,203,786,0.099656,0.098707,0.009577,0.000009
4,5,10.12,4.000,5.0,1012,209,803,0.102602,0.100841,0.017307,0.000030
5,6,9.67,5.000,6.0,967,196,771,0.096220,0.096823,-0.006246,0.000004
6,7,10.28,6.000,7.0,1028,177,851,0.086892,0.106869,-0.206935,0.004134
7,8,10.25,7.000,8.0,1025,197,828,0.096711,0.103981,-0.072482,0.000527
8,9,9.84,8.000,9.0,984,213,771,0.104566,0.096823,0.076931,0.000596
9,10,4.90,9.000,10.0,490,101,389,0.049583,0.048851,0.014869,0.000011


In [ ]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [ ]:
woetable=merge(woetable,1,2)

In [ ]:
woetable

,Bin Number,Percentage in Bin,Range,Total,No. of Good,No. of Bad,Dist. of Good,Dist. of Bad,WoE,IV
0,1,50.84,1,5084.0,1409.0,3675.0,0.691703,0.461509,0.404655,0.093149
1,2,49.16,2+3+4,4916.0,628.0,4288.0,0.308297,0.538491,-0.557708,0.128381


In [ ]:
def indexer (bin1, bin2,p):
  m
  index1=min(bin1,bin2)
  index2=max(bin1,bin2)
  print(index2)
  if (p==1):
    p1=mergeconti(p1,(index1-1),(index2-1))
  if (p==0):
    p1=merge()
  p1

6
7


,Bin Number,Percentage in Bin,Lower Limit,Upper Limit,Total,No. of Good,No. of Bad,Dist. of Good,Dist. of Bad,WoE,IV
0,1,10.20,17.999,27.0,1020.0,73.0,947.0,0.035837,0.118925,-1.199512,0.099665
1,2,13.52,27.000,31.0,1352.0,108.0,1244.0,0.053019,0.156223,-1.080628,0.111525
2,3,8.60,31.000,33.0,860.0,76.0,784.0,0.037310,0.098455,-0.970348,0.059332
3,4,9.21,33.000,35.0,921.0,90.0,831.0,0.044183,0.104358,-0.859492,0.051720
4,5,9.34,35.000,37.0,934.0,115.0,819.0,0.056456,0.102851,-0.599824,0.027829
5,6,20.19,37.000,42.0,2019.0,370.0,1649.0,0.181640,0.207083,-0.131094,0.003335
6,7,10.09,42.000,46.0,1009.0,339.0,670.0,0.166421,0.084139,0.682050,0.056120
7,8,9.19,46.000,53.0,919.0,473.0,446.0,0.232204,0.056009,1.422104,0.250568
8,9,9.66,53.000,92.0,966.0,393.0,573.0,0.192931,0.071958,0.986252,0.119310


In [ ]:
import plotly.graph_objs as go
import plotly
import plotly.figure_factory as ff
from plotly.offline import iplot

In [ ]:
def plotter(df):
  data = [go.Bar(x=df['Bin Number'],
                 y=df['WoE'],
                 text=df['Bin Number'],
                 marker=dict(
                 color='rgb(158,20,25)',
                 line=dict(
                 color='rgb(8,48,107)',
                 width=1.5,)
                 ),opacity=0.6
                 )]
  layout = go.Layout(
      title='Weight of Evidence(WOE) Table',
      xaxis=dict(
      title='Bin Number',
      tickangle=-45,
      tickfont=dict(
      size=10,
      color='rgb(107, 107, 107)'
      )
      ),
      yaxis=dict(
          title='Weight of Evidence(WOE)',
          titlefont=dict(
          size=14,
          color='rgb(107, 107, 107)'
          ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)')),)
  plotly.offline.iplot({
      "data": data,'layout':layout})

In [ ]:
plotter(p1)

In [ ]:
feature='Geography'
nrow=len(df.index)
woerow=len(woetable.index)
z=list()
i=0
while (i<woerow):
  if (type(woetable.loc[i].get(key='Range'))==np.float64) or (type(woetable.loc[i].get(key='Range'))==np.int64) or (type(woetable.loc[i].get(key='Range'))==int):
    z.append(repr(woetable.loc[i].get(key='Range')))
  else:
    z.append(woetable.loc[i].get(key='Range'))
  i=i+1
woetable['Range']=z
i=0
while(i<woerow):
  x=0
  range=woetable.loc[i].get(key='Range')
  if(range.find('+')!=-1):
    x=1
    p=range.split('+')
    print(p)
  i=i+1

['Germany', 'Spain']
